In [2]:
import ee
import geemap

ee.Initialize()

region = ee.Geometry.Rectangle([12,42,13,43]) 


Map = geemap.Map(center=(42, 12), zoom=10)
Map.addLayer(region)
Map

Map(center=[42, 12], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=HBox(children=(Tog…

In [3]:
def get_best_s2_image(aoi, start_date, end_date):
    # Import and filter S2 SR.
    s2_sr_col = (ee.ImageCollection('COPERNICUS/S2_SR')
                 .filterBounds(aoi)
                 .filterDate(start_date, end_date))
                 #.filter(ee.Filter.lte('CLOUDY_PIXEL_PERCENTAGE', CLOUD_FILTER)))

    # Import and filter s2cloudless.
    s2_cloudless_col = (ee.ImageCollection('COPERNICUS/S2_CLOUD_PROBABILITY')
                        .filterBounds(aoi)
                        .filterDate(start_date, end_date))

    # Join the filtered s2cloudless collection to the SR collection by the 'system:index' property.
    imgs = ee.ImageCollection(ee.Join.saveFirst('s2cloudless').apply(**{
        'primary': s2_sr_col,
        'secondary': s2_cloudless_col,
        'condition': ee.Filter.equals(**{
            'leftField': 'system:index',
            'rightField': 'system:index'
        })
    }))

    def get_clouds_per(img):

        img = ee.Image(img.get('s2cloudless'))

        eo = ee.Dictionary(img.reduceRegion(ee.Reducer.median(), aoi, 100))

        return ee.Feature(None, {'prob': eo.get('probability')})

    if imgs.size().getInfo() > 0:
        # Get the best image id sorting respect to the cloud %
        best_image_id = imgs.map(get_clouds_per).sort('prob').first().id().getInfo()

        best_image = ee.Image("COPERNICUS/S2_SR/" + best_image_id).addBands(
            ee.Image("COPERNICUS/S2_CLOUD_PROBABILITY/" + best_image_id))

        return best_image
    else:
        return None


Ciao Notebook


In [ ]:
t0 = ee.Date(2020-03-01)
t1 = ee.Date(2020-06-01)

aoi = ee.Geometry.Rectangle([12,42,13,43]) 